<a href="https://colab.research.google.com/github/Karagiannis-Giorg/A-new-perspective-to-Smart-Fridges/blob/main/Simulation/Simulation_Y_F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install roboflow
!pip install ultralytics==8.3.40
!pip install streamlit ngrok opencv-python-headless
!pip install pyngrok
!ngrok config add-authtoken 2rXJVXzr26fTTIW0Qj1torNd6Il_iiVaeFHCJtw5zki9ubVc

import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]

from detectron2.utils.logger import setup_logger
setup_logger()

from google.colab import drive
drive.mount('/content/gdrive')
ROOT_DIR = '/content/gdrive/My Drive/Georgios_Karagiannis/Code'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-nm_5xpgp
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-nm_5xpgp
  Resolved https://github.com/facebookresearch/detectron2.git to commit 9604f5995cc628619f0e4fd913453b4d7d61db3f
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 81.5 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp3

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="jrkIR8tZdg5Y8uIaKmwZ")
project = rf.workspace("smart-fridge-jvm9v").project("smart-fridge-p5rw0")
version = project.version(9)
dataset = version.download("coco")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Smart-Fridge-9 in coco:: 100%|██████████| 1175/1175 [00:00<00:00, 3738.19it/s]


In [ ]:
%%writefile smart_fridge_app.py
import streamlit as st
import numpy as np
import glob
import cv2
import random
from google.colab.patches import cv2_imshow
import os
import shutil
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
from ultralytics import YOLO
import json

# Καταχώριση των δεδομένων COCO και προσθήκη των κατηγοριών
def register_coco_instances_with_classes(dataset_name, json_file, image_dir):
    # Εγγραφή των δεδομένων COCO
    register_coco_instances(dataset_name, {}, json_file, image_dir)

    # Ανάγνωση του αρχείου json για να πάρουμε τις κατηγορίες
    with open(json_file, 'r') as f:
        coco_data = json.load(f)

    # Λήψη των κατηγοριών από το αρχείο COCO
    thing_classes = [category['name'] for category in coco_data['categories']]

    # Ενημέρωση του metadata για να περιλαμβάνει τις κατηγορίες
    metadata = MetadataCatalog.get(dataset_name)
    metadata.thing_classes = thing_classes

# Εγγραφή των δεδομένων για train, val, test και ενημέρωση των κατηγοριών
register_coco_instances_with_classes("my_dataset_train", "/content/Smart-Fridge-9/train/_annotations.coco.json", "/content/Smart-Fridge-9/train")

# Ρύθμιση Faster R-CNN
def setup_faster_rcnn():
    cfg = get_cfg()
    cfg.merge_from_file("/content/gdrive/MyDrive/Georgios_Karagiannis/Code/config.yml")
    cfg.MODEL.WEIGHTS = "/content/gdrive/MyDrive/Georgios_Karagiannis/Code/D-level2/model_final.pth"
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.75
    predictor = DefaultPredictor(cfg)
    metadata = MetadataCatalog.get("my_dataset_train")
    return predictor, metadata

# Ρύθμιση YOLO
def setup_yolo():
    model = YOLO("/content/gdrive/My Drive/Georgios_Karagiannis/Code/Level1_train/detect/train/weights/best.pt")
    return model

# Ανάλυση Εικόνας
def analyze_image(image, temperature, humidity, predictor, metadata, yolo_model):
    yolo_results = yolo_model(image)
    fruit_classes = [yolo_model.names[int(cls)] for cls in yolo_results[0].boxes.cls]

    if "Tomato" in fruit_classes:
        im = cv2.imread(image)
        outputs = predictor(im)
        pred_classes_indices = outputs["instances"].pred_classes.cpu().numpy()
        ripeness_classes = [metadata.thing_classes[i] for i in pred_classes_indices]
        ripeness_message = []

        for ripeness in ripeness_classes:
            if ripeness == "over-ripe":
                ripeness_message.append("Over-ripe tomatoes detected. Discard them immediatley before they start to effect the other fruits.")
            elif ripeness == "ripe":
                if 7 <= temperature <= 10 and humidity > 70:
                    ripeness_message.append("Ripe tomatoes detected. They are edible for up to 4 days. If you want your tomatoes to last longer, you should store them somewhere cooler.")
                elif temperature < 7:
                    ripeness_message.append("You should put your ripe tomatoes somewhere warmer to avoid chilling injuries.")
                else:
                    ripeness_message.append("YYour tomatoes will ripen quickly, but you should bear in mind that they are susceptible to pathogens.")
            elif ripeness == "half-ripe":
                if 8 <= temperature <= 10 and humidity > 60:
                    ripeness_message.append("Half-ripe tomatoes detected. They are not edible but can be stored for up to 7 days untill they are edible.If you want to ripe them more slower , you should store them somewhere cooler and with less humidity")
                elif temperature < 8:
                    ripeness_message.append("Your rippening prosses will be stoped and your tomatoes will be susceptible to chilling injuries.")
                else:
                    ripeness_message.append("Your tomatoes will ripen quickly, but you should bear in mind that they are susceptible to pathogens.")
            elif ripeness == "unripe":
                if 8 <= temperature <= 12 and humidity > 50:
                    ripeness_message.append("Unripe tomatoes detected. They are not edible but can be stored for up to 10 days untill they are edible.If you want to ripe them more slower , you should store them somewhere cooler and with less humidity")
                elif temperature < 8:
                    ripeness_message.append("Your rippening prosses will be stoped and your tomatoes will be susceptible to chilling injuries.")
                else:
                    ripeness_message.append("Your tomatoes will ripen quickly, but you should bear in mind that they are susceptible to pathogens.")
        return ripeness_message
    else:
        remaining_items = [item for item in fruit_classes if item != "Tomato"]
        return f"No tomatoes detected. Items in the fridge: {', '.join(remaining_items)}"

# Streamlit UI
def main():
    # Κεντρικός τίτλος
    st.markdown(
        "<h1 style='text-align: center;'>Welcome back</h1>",
        unsafe_allow_html=True,
    )
    # Εισαγωγή δεδομένων χρήστη
    temperature = st.number_input(
        "Temperature (°C)", value=8.0, min_value=-10.0, max_value=15.0, step=1.0
    )
    humidity = st.number_input(
        "Humidity (%)", value=75.0, min_value=0.0, max_value=100.0, step=1.0
    )

    st.markdown(
        "<h3 style='text-align: center;'>What would you like to do?</h3>",
        unsafe_allow_html=True,
    )

    # Αρχικοποίηση Session State για την εικόνα
    if "selected_image" not in st.session_state:
        simulation_image_path = "/content/gdrive/MyDrive/Georgios_Karagiannis/Code/simulation"
        image_files = glob.glob(os.path.join(simulation_image_path, "*.[jp][pn]g"))
        if not image_files:
            st.error("No images found in the test directory.")
            st.stop()
        st.session_state.selected_image = random.choice(image_files)

    selected_image = st.session_state.selected_image

    if st.button("See what's inside"):
        # Εκτέλεση YOLO για ανάλυση περιεχομένων ψυγείου
        predictor, metadata = setup_faster_rcnn()
        yolo_model = setup_yolo()

        # Διαβάζουμε την εικόνα
        img = cv2.imread(selected_image)

        # Εκτελούμε το μοντέλο YOLO στην εικόνα
        results = yolo_model([img])

        # Σχεδιάζουμε τα αποτελέσματα πάνω στην εικόνα
        for result in results:
            result.save(filename="output.jpg")  # Αποθηκεύουμε την έξοδο σε αρχείο

        # Ανάλυση φρούτων
        ripeness_analysis = analyze_image(selected_image, temperature, humidity, predictor, metadata, yolo_model)

        # Εμφάνιση νέας ανάλυσης
        st.success("Analysis Complete!")
        st.image("output.jpg", caption="Detected Items in the Fridge", use_container_width=True)
        fruit_classes = [yolo_model.names[int(cls)] for cls in results[0].boxes.cls]
        fruits_counts = {fruits: fruit_classes.count(fruits) for fruits in set(fruit_classes)}

        st.write("Contents of the fridge:")
        for fruits, count in fruits_counts.items():
            st.write(f"- {count} {fruits}(s)")

    if st.button("See the maturity"):
        # Εκτέλεση Faster R-CNN για ανάλυση περιεχομένων ψυγείου
        predictor, metadata = setup_faster_rcnn()
        yolo_model = setup_yolo()
        im = cv2.imread(selected_image)

        # Εκτέλεση πρόβλεψης
        outputs = predictor(im)

        # Οπτικοποίηση αποτελεσμάτων με Visualizer
        v = Visualizer(
            im[:, :, ::-1],  # Convert BGR to RGB
            metadata=metadata,
            scale=0.8
        )
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

        # Μετατροπή της εικόνας από OpenCV (numpy array) σε μορφή PNG
        processed_img_path = "fridge_analysis.png"
        cv2.imwrite(processed_img_path, out.get_image()[:, :, ::-1])  # Convert RGB to BGR for OpenCV

        st.success("Analysis Complete!")
        # Εμφάνιση της εικόνας με ανιχνεύσεις
        st.image(processed_img_path, caption="Ripeness of tomatoes detected", use_container_width=True)

        # Ανάλυση ωριμότητας
        ripeness_analysis = analyze_image(selected_image, temperature, humidity, predictor, metadata, yolo_model)

        # Εμφάνιση ανάλυσης ωριμότητας
        if isinstance(ripeness_analysis, list):
            st.write("Ripeness Results:")
            for msg in ripeness_analysis:
                st.write("- " + msg)
        else:
            st.write(ripeness_analysis)

if __name__ == "__main__":
    main()

Writing smart_fridge_app.py


In [ ]:
from pyngrok import ngrok
import os

# Εκκίνηση Streamlit server
os.system("streamlit run smart_fridge_app.py &")

# Σύνδεση με ngrok
public_url = ngrok.connect(8501)  # Η θύρα πρέπει να είναι αριθμός χωρίς `port=`
print(f"Public URL: {public_url}")

Public URL: NgrokTunnel: "https://d22d-35-227-139-211.ngrok-free.app" -> "http://localhost:8501"


# Αξιολόγηση του Pipeline

In [ ]:
# Δεδομένα σε yolo11 + coco format για την αξιολόγηση
from roboflow import Roboflow
rf = Roboflow(api_key="F357b98Kl2koA8RqsGzW")
project = rf.workspace("simulation-sv1vx").project("simulation-dsrze")
version = project.version(2)
dataset = version.download("yolov11")



rf = Roboflow(api_key="F357b98Kl2koA8RqsGzW")
project = rf.workspace("simulation-sv1vx").project("simulation-repenning")
version = project.version(2)
dataset = version.download("coco")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Simulation-2 in yolov11:: 100%|██████████| 52/52 [00:00<00:00, 7484.94it/s]


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Simulation-repenning-2 in coco:: 100%|██████████| 27/27 [00:00<00:00, 5243.12it/s]


In [ ]:
import numpy as np
import glob
import random
import os
import shutil

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
from ultralytics import YOLO
import json


# Καταχώριση των δεδομένων COCO και προσθήκη των κατηγοριών
def register_coco_instances_with_classes(dataset_name, json_file, image_dir):
    # Εγγραφή των δεδομένων COCO
    register_coco_instances(dataset_name, {}, json_file, image_dir)

    # Ανάγνωση του αρχείου json για να πάρουμε τις κατηγορίες
    with open(json_file, 'r') as f:
        coco_data = json.load(f)

    # Λήψη των κατηγοριών από το αρχείο COCO
    thing_classes = [category['name'] for category in coco_data['categories']]

    # Ενημέρωση του metadata για να περιλαμβάνει τις κατηγορίες
    metadata = MetadataCatalog.get(dataset_name)
    metadata.thing_classes = thing_classes

# Εγγραφή των δεδομένων για train, val, test και ενημέρωση των κατηγοριών
register_coco_instances_with_classes("my_dataset_train", "/content/Smart-Fridge-9/train/_annotations.coco.json", "/content/Smart-Fridge-9/train")
register_coco_instances_with_classes("my_dataset_test", "/content/Simulation-repenning-2/valid/_annotations.coco.json", "/content/Simulation-repenning-2/valid")

# Ρύθμιση Faster R-CNN
def setup_faster_rcnn():
    cfg = get_cfg()
    cfg.merge_from_file("/content/gdrive/MyDrive/Georgios_Karagiannis/Code/config.yml")
    cfg.MODEL.WEIGHTS = "/content/gdrive/MyDrive/Georgios_Karagiannis/Code/D-level2/model_final.pth"
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.75
    predictor = DefaultPredictor(cfg)
    metadata = MetadataCatalog.get("my_dataset_train")
    return predictor, metadata, cfg

predictor, metadata, cfg = setup_faster_rcnn()
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Φόρτωση YOLO μοντέλου και αξιολόγηση
def evaluate_yolo(model_path, dataset_path):
    model = YOLO(model_path)
    results = model.val(data=dataset_path)  # Αξιολόγηση
    return results.box.maps  # Επιστροφή mAP τιμών

# Φόρτωση Faster R-CNN και αξιολόγηση
def evaluate_faster_rcnn(cfg, dataset_name):
    evaluator = COCOEvaluator(dataset_name, output_dir="./output")
    val_loader = build_detection_test_loader(cfg, dataset_name)
    results = inference_on_dataset(DefaultPredictor(cfg).model, val_loader, evaluator)
    return results

# Υπολογισμός συνολικής ακρίβειας
# (προσαρμόζεται ανάλογα με τη χρήση των μοντέλων)
def compute_overall_accuracy(yolo_metrics, faster_rcnn_metrics):
    yolo_map = yolo_metrics[1]*100 # mAP@50 του YOLO
    faster_rcnn_map = faster_rcnn_metrics["bbox"]['AP']  # mAP του Faster R-CNN
    print(f"YOLO mAP: {yolo_map}")
    print(f"Faster R-CNN mAP: {faster_rcnn_map}")
    overall_accuracy = (yolo_map + faster_rcnn_map) / 2  # Μέσος όρος
    return overall_accuracy

yolo_results=evaluate_yolo("/content/gdrive/My Drive/Georgios_Karagiannis/Code/Level1_train/detect/train/weights/best.pt", "/content/Simulation-2/data.yaml")

fast_rcnn_inferences=evaluate_faster_rcnn(cfg, "my_dataset_test")

overall_accuracy = compute_overall_accuracy(yolo_results, fast_rcnn_inferences)
print(f"Overall Accuracy: {overall_accuracy}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
[02/13 11:06:59 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/gdrive/MyDrive/Georgios_Karagiannis/Code/D-level2/model_final.pth ...
Ultralytics 8.3.40 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 238 layers, 2,583,712 parameters, 0 gradients, 6.3 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 91.3MB/s]
val: Scanning /content/Simulation-2/valid/labels... 23 images, 0 backgrounds, 0 corrupt: 100%|██████████| 23/23 [00:00<00:00, 821.01it/s]

val: New cache created: /content/Simulation-2/valid/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:03<00:00,  1.64s/it]


                   all         23        104      0.952      0.967      0.993      0.923
                 Apple         13         15      0.929          1      0.995      0.946
                Banana         10         10      0.958        0.9      0.978      0.825
                  Lime          7          7       0.98          1      0.995      0.938
                Orange          5          5      0.932          1      0.995      0.976
                 Peach          6          6      0.858          1      0.995      0.947
                  Pear          3          3      0.964          1      0.995      0.865
           Pomegranate          9          9          1      0.864      0.995      0.957
                Tomato         22         49          1      0.972      0.995      0.931
Speed: 4.8ms preprocess, 16.5ms inference, 0.0ms loss, 18.2ms postprocess per image
Results saved to runs/detect/val
WARNING [02/13 11:07:25 d2.data.datasets.coco]: 
Category ids in annotations are n

torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)


[02/13 11:07:29 d2.evaluation.evaluator]: Inference done 11/23. Dataloading: 0.0012 s/iter. Inference: 0.0948 s/iter. Eval: 0.0002 s/iter. Total: 0.0962 s/iter. ETA=0:00:01
[02/13 11:07:30 d2.evaluation.evaluator]: Total inference time: 0:00:01.915208 (0.106400 s / iter per device, on 1 devices)
[02/13 11:07:30 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:01 (0.101331 s / iter per device, on 1 devices)
[02/13 11:07:30 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[02/13 11:07:30 d2.evaluation.coco_evaluation]: Saving results to ./output/coco_instances_results.json
[02/13 11:07:30 d2.evaluation.coco_evaluation]: Evaluating predictions with unofficial COCO API...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
[02/13 11:07:30 d2.evaluation.fast_eval_api]: Evaluate annotation type *bbox*
[02/13 11:07:30 d2.evaluation.fast_eval_api]: COCOeval_opt.evaluate() finished in 0.01 seconds.
[02/13 11:07:30 d2.evaluation.